In [ ]:
import urllib
import requests
import time
import os

headers = {
    'Content-Type': 'application/x-www-form-urlencoded'
}

tenant_id = os.environ.get('DATABRICKS_TENANT', 'Common')
client_id = os.environ.get('DATABRICKS_CLINET_ID')
code_url = f'https://login.microsoftonline.com/{tenant_id}/oauth2/v2.0/devicecode'
token_url = f'https://login.microsoftonline.com/{tenant_id}/oauth2/v2.0/token'
scope = urllib.parse.quote('https://storage.azure.com/.default offline_access')
client_body = f'client_id={client_id}&scope={scope}'

print(f'Authenticating to: {code_url}')
code_response = requests.post(code_url, data=client_body, headers=headers).json()
print(code_response.get('message'))

grant_type = urllib.parse.quote('urn:ietf:params:oauth:grant-type:device_code')
device_code = code_response.get('device_code')
token_body = f'grant_type={grant_type}&client_id={client_id}&device_code={device_code}'



In [ ]:
expires_in = code_response.get('expires_in')
while expires_in > 0:
    token_response = requests.post(token_url, data=token_body, headers=headers).json()
    if token_response.get('refresh_token') is not None:
        break
    time.sleep(5)
    expires_in -= 5

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
spark.conf.set('fs.azure.account.auth.type', 'OAuth')
spark.conf.set('fs.azure.account.oauth.provider.type', 'org.apache.hadoop.fs.azurebfs.oauth2.RefreshTokenBasedTokenProvider')
spark.conf.set('fs.azure.account.oauth2.refresh.token', token_response.get('refresh_token'))
spark.conf.set('fs.azure.account.oauth2.refresh.endpoint', 'https://login.microsoftonline.com/Common/oauth2/token')
spark.conf.set('fs.azure.account.oauth2.client.id', client_id)
